# TypeScript Support

This notebook demonstrates how to use TypeScript in lyric.

In [1]:
from lyric import DefaultLyricDriver

lcd = DefaultLyricDriver(host="127.0.0.1", log_level="ERROR")
lcd.start()

# Load workers(default: Python, JavaScript)
# await lcd.lyric.load_default_workers() // Not necessary

## Install lyric-component-ts-transpiling

In lyric, typescript is not supported directly. This package is a component that transpiles typescript to javascript.

```bash
pip install lyric-component-ts-transpiling
```

In [2]:
from lyric_component_ts_transpiling import TypeScriptWasmTaskSpec
from lyric.py_lyric import TypeScriptComponentLoader, Language, JavaScriptWorkerLoader

ts_worker_name = "ts_worker"
# Register javascript worker
lcd.lyric.register_worker(ts_worker_name, Language.JAVASCRIPT, JavaScriptWorkerLoader())

In [3]:
# Load javascript worker with TypeScriptComponentLoader as dependencies
await lcd.lyric.load_worker(ts_worker_name, dependencies=[TypeScriptComponentLoader()])

HandleItem(worker_info=WorkerInfo(name='ts_worker', lang=<Language.JAVASCRIPT: 1>, loader=<lyric.py_lyric.JavaScriptWorkerLoader object at 0x10f6cdf00>, dependencies=[]), handle=<builtins.PyTaskHandle object at 0x10f6ce070>, environment_config=<builtins.PyEnvironmentConfig object at 0x10f5f9570>, dependencies=[<builtins.PyTaskHandle object at 0x10f689da0>])

In [4]:
ts_code = """
interface Point {
    x: number;
    y: number;
}

function calculateDistance(p1: Point, p2: Point): number {
    const dx = p2.x - p1.x;
    const dy = p2.y - p1.y;
    return Math.sqrt(dx * dx + dy * dy);
}


const point1: Point = { x: 0, y: 0 };
const point2: Point = { x: 3, y: 4 };

const distance = calculateDistance(point1, point2);
console.log(`The distance between the two points is ${distance}`);
"""

js_res = await lcd.exec(ts_code, "typescript", worker_name=ts_worker_name)
print(js_res)

["local_798ab628d0ec0ba3@35759259-4ad8-4aef-b827-0a407980b802"]: "[JavaScript-InterpreterTask] script: {\"protocol\":1,\"lang\":\"TYPESCRIPT\",\"code\":\"\\ninterface Point {\\n    x: number;\\n    y: number;\\n}\\n\\nfunction calculateDistance(p1: Point, p2: Point): number {\\n    const dx = p2.x - p1.x;\\n    const dy = p2.y - p1.y;\\n    return Math.sqrt(dx * dx + dy * dy);\\n}\\n\\n\\nconst point1: Point = { x: 0, y: 0 };\\nconst point2: Point = { x: 3, y: 4 };\\n\\nconst distance = calculateDistance(point1, point2);\\nconsole.log(`The distance between the two points is ${distance}`);\\n\"}"
["local_798ab628d0ec0ba3@35759259-4ad8-4aef-b827-0a407980b802"]: "Transpiling TypeScript code to JavaScript"
["local_798ab628d0ec0ba3@35759259-4ad8-4aef-b827-0a407980b802"]: "Transpiled TypeScript code to JavaScript: function calculateDistance(p1, p2) {"
["local_798ab628d0ec0ba3@35759259-4ad8-4aef-b827-0a407980b802"]: "  const dx = p2.x - p1.x;"
["local_798ab628d0ec0ba3@35759259-4ad8-4aef-b827-

In [5]:
import json

ts_func = """
// Use an index signature to define a message dictionary that can contain any fields
interface MessageDictionary {
    user_message: string;      // Keep the required fields
    ai_message: string;        // Keep the required fields
    [key: string]: string;     // Allow any other fields
}

interface MessageHandlerResult {
    user: string;
    ai: string;
    all: string[];
    custom: string;
    handler_language: string;
}

function message_handler(message_dict: MessageDictionary): MessageHandlerResult {
    return {
        user: message_dict.user_message,
        ai: message_dict.ai_message,
        all: [message_dict.user_message, message_dict.ai_message],
        custom: "custom",
        handler_language: "javascript",
    };
}
"""

# Create a input dictionary
input_data = {
    "user_message": "Hello from user",
    "ai_message": "Hello from AI",
}
# Convert the input dictionary to json bytes, as the input to the function(It will be converted to dictionary inside the function)
input_bytes = json.dumps(input_data).encode("utf-8")

ts_func_res = await lcd.exec1(ts_func, input_bytes, "message_handler", lang="typescript", worker_name=ts_worker_name)
print("TypeScript function result:", ts_func_res)
print("Full output:", ts_func_res)

["local_798ab628d0ec0ba3@35759259-4ad8-4aef-b827-0a407980b802"]: "[JavaScript-InterpreterTask] script: {\"protocol\":1,\"lang\":\"TYPESCRIPT\",\"code\":\"\\n// Use an index signature to define a message dictionary that can contain any fields\\ninterface MessageDictionary {\\n    user_message: string;      // Keep the required fields\\n    ai_message: string;        // Keep the required fields\\n    [key: string]: string;     // Allow any other fields\\n}\\n\\ninterface MessageHandlerResult {\\n    user: string;\\n    ai: string;\\n    all: string[];\\n    custom: string;\\n    handler_language: string;\\n}\\n\\nfunction message_handler(message_dict: MessageDictionary): MessageHandlerResult {\\n    return {\\n        user: message_dict.user_message,\\n        ai: message_dict.ai_message,\\n        all: [message_dict.user_message, message_dict.ai_message],\\n        custom: \\\"custom\\\",\\n        handler_language: \\\"javascript\\\",\\n    };\\n}\\n\"}"
["local_798ab628d0ec0ba3@357592

In [6]:
ts_func_res.output

{'ai': 'Hello from AI',
 'all': ['Hello from user', 'Hello from AI'],
 'custom': 'custom',
 'handler_language': 'javascript',
 'user': 'Hello from user'}